<a href="https://colab.research.google.com/github/sezinbiner/find_path_genetic_algorithms/blob/main/find_pathv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import math
import random
import copy

moves=[[-1,0], [-1,-1], [0, -1], [1,-1], [1,0], [1, 1], [0, 1], [-1, 1]]
directions = [1,2,3,4,5,6,7,8];

def find_cost(old_move, new_move):
  if abs(old_move - new_move) == 1 or abs(old_move - new_move) == 7:
    return 1
  elif abs(old_move - new_move) == 2 or abs(old_move - new_move) == 6:
    return 2
  elif abs(old_move - new_move) == 3 or abs(old_move - new_move) == 5:
   return 3
  elif abs(old_move - new_move) == 0:
    return 0
  else:
    return 4

def find_way(end):
  if end[0] == 0 and end[1] == 0:
      return random.randint(5,7)
  elif end[0] == 8 and end[1] == 8:
    return random.randint(1,3)
  elif end[0] == 8 and end[1] == 0:
    return random.choice([1,7,8])
  elif end[0] == 0 and end[1] == 8:
    return random.randint(3,5)
  elif end[0] == 8:
    return random.choice([3,2,1,7,8])
  elif end[0] == 0:
    return random.choice([3,4,5,6,7])
  elif end[1] == 8:
    return random.choice([1,2,3,4,5])
  elif end[1] == 0:
    return random.choice([5,6,7,8,1])
  else:
    return random.randint(1,8)


def create_first_route(start):
  fitness_list = []
  routes = []
  
  for i in range (population):
    matrix = np.zeros((size,size))
    matrix[start[0], start[1]] = 1;
    end = copy.deepcopy(start)
    route = []
    cost = 0
    route.append(1)
    while len(route) <= 80:
      route.append(find_way(end))
      cost += find_cost(route[-2], route[-1])
      add_move = moves[route[-1]-1]
      end[0] += add_move[0]
      end[1] += add_move[1]
      matrix[end[0], end[1]] = 1
    
    area = len(np.argwhere(matrix == 1))
    routes.append(route)
    fitness_list.append(find_fitness(start, end, cost, area))
  return routes, fitness_list

def try_routes(coming_routes, start, size):
  fitness_list = []
  return_routes = []
  end=[0,0]
  
  for route in coming_routes: 
    flag=0
    cost = 0            #population 5000
    matrix = np.zeros((size,size))
    matrix[start[0], start[1]] = 1
    end[0]=start[0]
    end[1]=start[1]
    for i in range (len(route)):       #length of one route 9*9-1
      if (i !=0):
        add_move =  moves[route[i]-1]
        end = [end[0] + add_move[0], end[1] + add_move[1]]
        if (end[0] <= 8) and (end[1] <= 8) and (end[0] >= 0) and (end[1] >= 0): #kaç kişi siliniyor bak az silinen varsa create route
          cost += find_cost(route[i-1], route[i])
          matrix[end[0], end[1]] = 1
        else:  
          end = [end[0] - add_move[0], end[1] - add_move[1]]
          route[i]=find_way(end)
          add_move =moves[route[i]-1]
          end = [end[0] + add_move[0], end[1] + add_move[1]]
          cost += find_cost(route[i-1], route[i])
          matrix[end[0], end[1]] = 1  
    
    area = len(np.argwhere(matrix == 1))
    return_routes.append(route)
    fitness_list.append(find_fitness(start, end, cost, area))
  return return_routes,fitness_list

def calculate_distance(start,end):
    dist = math.sqrt(((start[0]-end[0])**2)+((start[1]-end[1])**2))
    return dist

def find_fitness(start, end, cost, area):
  distance = calculate_distance(start, end)
  fitness = (2*area) - cost - (4*distance)
  return fitness

def cross_over(routes):
  new_generation = []

  count=0
  for i in range (len(routes)):

    route1 = random.choice(routes)
    route2 = random.choice(routes)
    #new_route1 = route1[0:21] + route2[21:41] + route1[41:61] + route2[61:]
    #new_route2 = route2[:21] + route1[21:41] + route2[41:61] + route1[61:]
    new_route1 = route1[:1]+route1[1:41] + route2[41:] 
    new_route2 = route2[:1]+route2[1:41] + route1[41:]      

    new_generation.append(new_route1)
    new_generation.append(new_route2)

  return new_generation

def mutation(routes, mutation_ratio, length, population):

  for i in range(int(mutation_ratio)):
    index = random.randint(0,population-1)
    route = routes[index]        #route sayısı çok azaldığı için hata alıyoruz
    for i in range (3):
      index2 = random.randint(1,length)
      direction = random.randint(1,8)
      route[index2] = direction
    routes[index] = route
  return routes

size = 9
population = 5000
selected = int(population/2)
generation = 200
length = size*size-1
mutation_ratio = 0.01

start = [5,5]
end = copy.deepcopy(start)

routes, fitness_list = create_first_route(start)
zipped = list(zip(routes, fitness_list))
sorted_routes = sorted(zipped, key = lambda x: x[1])
unzipped_list = list(zip(*sorted_routes))
new_generation = unzipped_list[0]

for i in range (generation-1):
  cross_generation = cross_over(new_generation[selected:])   
  #print("new gen:",len(cross_generation))
  mutated_generation = mutation(cross_generation, mutation_ratio*population, length, population)
  #print("muntanted gen:",len(mutated_generation))  
  new_routes, new_fitness = try_routes(mutated_generation, start, size)
  #print("new route_list lenght:", len(new_routes))
  zipped = list(zip(new_routes, new_fitness))
  sorted_routes = sorted(zipped, key = lambda x: x[1])
  unzipped_list = list(zip(*sorted_routes))
  new_generation = unzipped_list[0]
  print("döngü",i+1)
print("bitti")



döngü 1
döngü 2
döngü 3
döngü 4
döngü 5
döngü 6
döngü 7
döngü 8
döngü 9
döngü 10
döngü 11
döngü 12
döngü 13
döngü 14
döngü 15
döngü 16
döngü 17
döngü 18
döngü 19
döngü 20
döngü 21
döngü 22
döngü 23
döngü 24
döngü 25
döngü 26
döngü 27
döngü 28
döngü 29
döngü 30
döngü 31
döngü 32
döngü 33
döngü 34
döngü 35
döngü 36
döngü 37
döngü 38
döngü 39
döngü 40
döngü 41
döngü 42
döngü 43
döngü 44
döngü 45
döngü 46
döngü 47
döngü 48
döngü 49
döngü 50
döngü 51
döngü 52
döngü 53
döngü 54
döngü 55
döngü 56
döngü 57
döngü 58
döngü 59
döngü 60
döngü 61
döngü 62
döngü 63
döngü 64
döngü 65
döngü 66
döngü 67
döngü 68
döngü 69
döngü 70
döngü 71
döngü 72
döngü 73
döngü 74
döngü 75
döngü 76
döngü 77
döngü 78
döngü 79
döngü 80
döngü 81
döngü 82
döngü 83
döngü 84
döngü 85
döngü 86
döngü 87
döngü 88
döngü 89
döngü 90
döngü 91
döngü 92
döngü 93
döngü 94
döngü 95
döngü 96
döngü 97
döngü 98
döngü 99
döngü 100
döngü 101
döngü 102
döngü 103
döngü 104
döngü 105
döngü 106
döngü 107
döngü 108
döngü 109
döngü 110
döngü 11

In [10]:
print(str(new_generation[0]))
print(len(new_generation[0]))

[1, 2, 1, 1, 1, 8, 6, 8, 7, 5, 5, 5, 5, 4, 1, 2, 4, 8, 7, 7, 2, 2, 8, 6, 3, 3, 5, 2, 3, 4, 6, 5, 5, 6, 7, 6, 1, 4, 5, 3, 8, 1, 1, 8, 5, 5, 2, 1, 1, 8, 6, 2, 6, 2, 6, 1, 4, 3, 3, 6, 1, 2, 2, 3, 2, 6, 6, 6, 6, 5, 6, 7, 1, 2, 1, 1, 1, 4, 4, 4, 6]
81


In [11]:
end[0]=start[0]
end[1]=start[1]
print("start:",start)
print("end",end)
matrix = np.zeros((size,size))
for i in range (len(new_generation[0])-1):
  add_move =moves[new_generation[0][i+1]-1]
  end = [end[0] + add_move[0], end[1] + add_move[1]]
  print("end:",end)
  matrix[end[0], end[1]] = 1
print(end)
area = len(np.argwhere(matrix == 1))
print(area)


start: [5, 5]
end [5, 5]
end: [4, 4]
end: [3, 4]
end: [2, 4]
end: [1, 4]
end: [0, 5]
end: [1, 6]
end: [0, 7]
end: [0, 8]
end: [1, 8]
end: [2, 8]
end: [3, 8]
end: [4, 8]
end: [5, 7]
end: [4, 7]
end: [3, 6]
end: [4, 5]
end: [3, 6]
end: [3, 7]
end: [3, 8]
end: [2, 7]
end: [1, 6]
end: [0, 7]
end: [1, 8]
end: [1, 7]
end: [1, 6]
end: [2, 6]
end: [1, 5]
end: [1, 4]
end: [2, 3]
end: [3, 4]
end: [4, 4]
end: [5, 4]
end: [6, 5]
end: [6, 6]
end: [7, 7]
end: [6, 7]
end: [7, 6]
end: [8, 6]
end: [8, 5]
end: [7, 6]
end: [6, 6]
end: [5, 6]
end: [4, 7]
end: [5, 7]
end: [6, 7]
end: [5, 6]
end: [4, 6]
end: [3, 6]
end: [2, 7]
end: [3, 8]
end: [2, 7]
end: [3, 8]
end: [2, 7]
end: [3, 8]
end: [2, 8]
end: [3, 7]
end: [3, 6]
end: [3, 5]
end: [4, 6]
end: [3, 6]
end: [2, 5]
end: [1, 4]
end: [1, 3]
end: [0, 2]
end: [1, 3]
end: [2, 4]
end: [3, 5]
end: [4, 6]
end: [5, 6]
end: [6, 7]
end: [6, 8]
end: [5, 8]
end: [4, 7]
end: [3, 7]
end: [2, 7]
end: [1, 7]
end: [2, 6]
end: [3, 5]
end: [4, 4]
end: [5, 5]
[5, 5]
39
